In [23]:
import sqlite3
import tweepy
import json
import sqlite3

In [17]:
consumer_key = "4yLfbC723iTMdpVuMG4sUIbgy"
consumer_secret = "WbztNPnzgWJNRpvwSuC6ViDyXNDwtjfKsSl5d6hBhbc1FDPE1t"
access_token = "955855581360910337-NxKQ23Q033lhuemZlFGyFUb5TaoiYqr"
access_token_secret = "qlqToJDCA9KgAHvuE8AnszPKf23MowZAHUk7xs0tVeBMQ"

In [18]:
# conn = sqlite3.connect('Tweets.db')
# c = conn.cursor()
# c.execute('''CREATE TABLE tweets
#     (tweetText text,
#     user text,
#     followers integer,
#     date text,
#     location text)''')
# conn.commit()
# conn.close()

In [19]:
global api

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# DB stuff
conn = sqlite3.connect('Tweets.db')
c = conn.cursor()

In [20]:
class Tweet():

    # Data on the tweet
    def __init__(self, text, user, followers, date, location):
        self.text = text
        self.user = user
        self.followers = followers
        self.date = date
        self.location = location

    # Inserting that data into the DB
    def insertTweet(self):

        c.execute("INSERT INTO tweets (tweetText, user, followers, date, location) VALUES (?, ?, ?, ?, ?)",
            (self.text, self.user, self.followers, self.date, self.location))
        conn.commit()

In [21]:
class TweetStreamListener(tweepy.StreamListener):

    # When data is received
    def on_data(self, data):

        # Error handling because teachers say to do this
        try:

            # Make it JSON
            tweet = json.loads(data)

            # filter out retweets
            if not tweet['retweeted'] and 'RT @' not in tweet['text']:

                # Get user via Tweepy so we can get their number of followers
                user_profile = api.get_user(tweet['user']['screen_name'])

                # assign all data to Tweet object
                tweet_data = Tweet(
                    str(tweet['text'].encode('utf-8')),
                    tweet['user']['screen_name'],
                    user_profile.followers_count,
                    tweet['created_at'],
                    tweet['user']['location'])

                # Insert that data into the DB
                tweet_data.insertTweet()
                print("Another sweet sweet tweet inserted into database")

        # Let me know if something bad happens
        except Exception as e:
            print(e)
            pass

        return True

In [ ]:
if __name__ == '__main__':

    # Run the stream!
    l = TweetStreamListener()
    stream = tweepy.Stream(auth, l)

    # Filter the stream for these keywords. Add whatever you want here!
    stream.filter(track=['Trump'])

Another sweet sweet tweet inserted into database
Another sweet sweet tweet inserted into database
Another sweet sweet tweet inserted into database
Another sweet sweet tweet inserted into database


In [8]:
import pandas as pd

In [9]:
df = pd.read_sql_query("select * from tweets limit 20000;", conn)

In [10]:
len(df)

250

In [14]:
df

,tweetText,user,followers,date,location
0,b'@SueNKGifTWriter Vous dite pamela et trump. ...,LeChapelierFou7,46,Mon Jul 22 22:48:18 +0000 2019,Dtc
1,b'@RyanAFournier Of course not. She\xe2\x80\x9...,Steve_Tatarunis,838,Mon Jul 22 22:48:18 +0000 2019,"New England, USA"
2,b'#ImpeachmentInquiryNow #ImpeachDonaldTrumpNOW',ladygingermint,114,Mon Jul 22 22:48:18 +0000 2019,"San Luis Obispo, CA"
3,b'@RealBasedMAGA @USAloveGOD @realDonaldTrump ...,sgrandmont,2606,Mon Jul 22 22:48:18 +0000 2019,None
4,b'@GentlemanGeorge @Jag11814459 @VeuveK @roseb...,JohnRya13591294,163,Mon Jul 22 22:48:18 +0000 2019,"Cúige Mumhan, Éire"
5,b'@AlbertoRodNews el se\xc3\xb1or Abrams le sa...,marcocol66,205,Mon Jul 22 22:48:18 +0000 2019,"Coral Springs, FL"
6,"b""OMG....wtf is wrong with people.....get over...",DeplorableSunny,7114,Mon Jul 22 22:48:18 +0000 2019,None
7,"b'honestly, this is like ppl who claim trump c...",ivysaintstyles,1530,Mon Jul 22 22:48:18 +0000 2019,"Bordéus, França"
8,b'#QAnon. Q stands for Quite the Moran',Scr0dman,48,Mon Jul 22 22:48:19 +0000 2019,FL
9,b'Siemens CEO says Trump is becoming the \xe2\...,cassady2euca,4960,Mon Jul 22 22:48:19 +0000 2019,"Columbus, Ohio"
